In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os

# modify the project path below accordingly
PATH = "/content/drive/MyDrive/HGR/gesture_face_features"

sys.path.append(os.path.join(PATH, "code"))

%load_ext autoreload
%autoreload 2

In [3]:
data_path = os.path.join(PATH, 'samples', 'sign')
feat_path = os.path.join(PATH, 'feats')

In [4]:
# from cam import *
from participant import Participant, load_participants
from loader import Loader, model_configs
from cam import Gradcam

In [5]:
loader = Loader(model_configs, data_path)
loader.load_models()
loader.load_dataset()

vgg19 loaded
resnet50 loaded
inception_v3 loaded
efficientnetb0 loaded
efficientnetb1 loaded
efficientnetb6 loaded
Found 206 files belonging to 7 classes.


In [6]:
p = load_participants(loader, feat_path)

Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_vgg19_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_resnet50_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_inception_v3_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_efficientnetb0_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_efficientnetb1_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p0_efficientnetb6_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p1_vgg19_feats.npz
Features already computed. Loading from /content/drive/MyDrive/HGR/gesture_face_features/feats/p1_resnet50_feats.npz
Features already computed. Loading from /content

In [7]:
!git clone https://github.com/thuml/LogME.git

fatal: destination path 'LogME' already exists and is not an empty directory.


In [8]:
from LogME.LogME import LogME

In [9]:
logme = LogME()

for i in range(5):
  print(f'LogME scores for participant {i}:')
  for model_name in loader.models:
    layer_name = loader.models[model_name][1][0]
    f = p[i].feats[model_name][layer_name]
    y = p[i].encode_labels()
    if len(f.shape) > 2:
      f = f.reshape(f.shape[0], -1)
    score = logme.fit(f, y)
    print(f'{model_name} {layer_name}: {score}.')
  print()

LogME scores for participant 0:
vgg19 block5_conv4: 19668.204914911235.


/content/LogME/LogME.py:172: UserWarning: re-fitting for new data. old parameters cleared.
  warnings.warn('re-fitting for new data. old parameters cleared.')


resnet50 conv5_block3_out: 17546.499380784146.
inception_v3 mixed10: 21067.12002232584.
efficientnetb0 top_conv: 11283.875357982173.
efficientnetb1 top_conv: 15043.912638734966.
efficientnetb6 top_conv: 132681.71211009988.

LogME scores for participant 1:
vgg19 block5_conv4: 21690.51194944081.
resnet50 conv5_block3_out: 19493.906044980507.
inception_v3 mixed10: 23923.952945414756.
efficientnetb0 top_conv: 12636.53893961318.
efficientnetb1 top_conv: 16788.476208310727.
efficientnetb6 top_conv: 147659.5101427307.

LogME scores for participant 2:
vgg19 block5_conv4: 21830.619459490168.
resnet50 conv5_block3_out: 19729.99091577155.
inception_v3 mixed10: 24352.40972747131.
efficientnetb0 top_conv: 12591.576931353595.
efficientnetb1 top_conv: 16887.027075485967.
efficientnetb6 top_conv: 147659.3889357587.

LogME scores for participant 3:
vgg19 block5_conv4: 25019.618095905356.
resnet50 conv5_block3_out: 22825.834797349984.
inception_v3 mixed10: 26794.848236835027.
efficientnetb0 top_conv: 14

In [10]:
f = p[5].feats[model_name][layer_name]
y = p[5].encode_labels()

# Recalculate the features and get (39,2304)
f.shape, y.shape

((40, 17, 17, 2304), (39,))

In [11]:
p = p[:5]

In [12]:
import numpy as np

# Initialize an empty dictionary to hold the stacked features and labels for each model_name and layer_name
all_p = {}

# Iterate through each object in p
for obj in p:
    for model_name in obj.feats:  # Loop through each model_name
        if model_name not in all_p:
            all_p[model_name] = {}  # Initialize an inner dictionary for model_name

        for layer_name in obj.feats[model_name]:  # Loop through each layer_name
            # Extract features (N, C) and labels (N,)
            features = obj.feats[model_name][layer_name]
            labels = obj.encode_labels()

            # If this layer_name is not in the inner dictionary, initialize the list
            if layer_name not in all_p[model_name]:
                all_p[model_name][layer_name] = ([], [])  # Initialize as empty lists (features, labels)

            # Append the current features and labels to the corresponding model_name -> layer_name
            all_p[model_name][layer_name][0].append(features)  # Append features
            all_p[model_name][layer_name][1].append(labels)    # Append labels

# Now, stack the features and labels for each (model_name, layer_name) pair
for model_name in all_p:
    for layer_name in all_p[model_name]:
        # Stack the features and labels for each layer_name under each model_name
        all_p[model_name][layer_name] = (
            np.vstack(all_p[model_name][layer_name][0]),  # Stack features along the N axis
            np.concatenate(all_p[model_name][layer_name][1])  # Concatenate labels along the N axis
        )


In [13]:
# for model_name in loader.models:
#   layer_name = loader.models[model_name][1][0]
#   f = all_p[model_name][layer_name][0]
#   y = all_p[model_name][layer_name][1]
#   if len(f.shape) > 2:
#     f = f.reshape(f.shape[0], -1)
#   score = logme.fit(f, y)
#   print(f'{model_name} {layer_name}: {score}.')
# print()

vgg19 block5_conv4: 4681.7274745225595.
resnet50 conv5_block3_out: 4060.5411517438847.
inception_v3 mixed10: 4853.370326863421.
efficientnetb0 top_conv: 2623.428730804909.
efficientnetb1 top_conv: 3537.1136151629476.
efficientnetb6 top_conv: 32212.91067477436.



In [14]:
!git clone https://github.com/google-research/google-research.git

fatal: destination path 'google-research' already exists and is not an empty directory.


In [18]:
import sys
sys.path.append('/content/google-research/stable_transfer/transferability')

In [19]:
!pip install ml_collections

In [21]:
%cd google-research
import logme

/content/google-research


In [23]:
# for i in range(5):
#   print(f'LogME scores for participant {i}:')
#   for model_name in loader.models:
#     layer_name = loader.models[model_name][1][1]
#     f = p[i].feats[model_name][layer_name]
#     y = p[i].encode_labels()
#     if len(f.shape) > 2:
#       f = f.reshape(f.shape[0], -1)
#     score = logme.get_logme_score(f, y.astype(np.int32))
#     print(f'{model_name} {layer_name}: {score}.')
#   print()

LogME scores for participant 0:
vgg19 fc2: 403.2294921875.
resnet50 avg_pool: 180.86605834960938.
inception_v3 avg_pool: 168.63465881347656.
efficientnetb0 avg_pool: 94.76738739013672.
efficientnetb1 avg_pool: 87.86012268066406.
efficientnetb6 avg_pool: 140.1593780517578.

LogME scores for participant 1:
vgg19 fc2: nan.
resnet50 avg_pool: 204.81077575683594.
inception_v3 avg_pool: 176.88717651367188.
efficientnetb0 avg_pool: 101.44648742675781.
efficientnetb1 avg_pool: 98.69146728515625.
efficientnetb6 avg_pool: 144.75460815429688.

LogME scores for participant 2:
vgg19 fc2: nan.
resnet50 avg_pool: 209.4711456298828.
inception_v3 avg_pool: 187.78085327148438.
efficientnetb0 avg_pool: 97.43122863769531.
efficientnetb1 avg_pool: 94.2161865234375.
efficientnetb6 avg_pool: 129.05825805664062.

LogME scores for participant 3:
vgg19 fc2: 523.2401123046875.
resnet50 avg_pool: 260.6561584472656.
inception_v3 avg_pool: 199.2091064453125.
efficientnetb0 avg_pool: 114.77733612060547.
efficientnet

KeyboardInterrupt: 

In [30]:
import gbc

for i in range(5):
  print(f'GBC scores for participant {i}:')
  for model_name in loader.models:
    layer_name = loader.models[model_name][1][1]
    f = p[i].feats[model_name][layer_name]
    y = p[i].encode_labels()
    if len(f.shape) > 2:
      f = f.reshape(f.shape[0], -1)
    score = gbc.get_gbc_score(f, y, 'spherical')
    print(f'{model_name} {layer_name}: {score}.')
  print()

LogME scores for participant 0:
vgg19 fc2: -0.0.
resnet50 avg_pool: -0.0.
inception_v3 avg_pool: -8.713795403594704e-29.
efficientnetb0 avg_pool: -2.0694946136542297e-15.
efficientnetb1 avg_pool: -2.694632680140731e-26.
efficientnetb6 avg_pool: -7.25595781627341e-33.

LogME scores for participant 1:
vgg19 fc2: -0.0.
resnet50 avg_pool: -0.0.
inception_v3 avg_pool: -0.0.
efficientnetb0 avg_pool: -7.669814743842978e-28.
efficientnetb1 avg_pool: -1.5735627398597585e-18.
efficientnetb6 avg_pool: -0.0.

LogME scores for participant 2:
vgg19 fc2: -0.0.
resnet50 avg_pool: -0.0.
inception_v3 avg_pool: -0.0.
efficientnetb0 avg_pool: -0.0.
efficientnetb1 avg_pool: -1.749143342643552e-36.
efficientnetb6 avg_pool: -0.0.

LogME scores for participant 3:
vgg19 fc2: -0.0.
resnet50 avg_pool: -0.0.
inception_v3 avg_pool: -0.0.
efficientnetb0 avg_pool: -1.7875197148529417e-27.
efficientnetb1 avg_pool: -5.685790326374109e-22.
efficientnetb6 avg_pool: -0.0.

LogME scores for participant 4:
vgg19 fc2: -0.0.

In [32]:
import hscore

for i in range(5):
  print(f'H scores for participant {i}:')
  for model_name in loader.models:
    layer_name = loader.models[model_name][1][1]
    f = p[i].feats[model_name][layer_name]
    y = p[i].encode_labels()
    if len(f.shape) > 2:
      f = f.reshape(f.shape[0], -1)
    score = hscore.get_hscore(f, y)
    print(f'{model_name} {layer_name}: {score}.')
  print()

LogME scores for participant 0:
vgg19 fc2: 0.629529595375061.
resnet50 avg_pool: 1.0204485654830933.
inception_v3 avg_pool: 0.45678049325942993.
efficientnetb0 avg_pool: 1.533376932144165.
efficientnetb1 avg_pool: 1.4832135438919067.
efficientnetb6 avg_pool: 1.0871301889419556.

LogME scores for participant 1:
vgg19 fc2: 0.8415642380714417.
resnet50 avg_pool: 1.1960978507995605.
inception_v3 avg_pool: 0.4509439766407013.
efficientnetb0 avg_pool: 2.0535876750946045.
efficientnetb1 avg_pool: 2.099924087524414.
efficientnetb6 avg_pool: 1.3438622951507568.

LogME scores for participant 2:
vgg19 fc2: 0.5934056043624878.
resnet50 avg_pool: 0.961187481880188.
inception_v3 avg_pool: 0.3832494020462036.
efficientnetb0 avg_pool: 1.560904622077942.
efficientnetb1 avg_pool: 1.5611777305603027.
efficientnetb6 avg_pool: 1.0211299657821655.

LogME scores for participant 3:
vgg19 fc2: 0.5489307641983032.
resnet50 avg_pool: 0.8596376180648804.
inception_v3 avg_pool: 0.32402241230010986.
efficientnetb0 

In [33]:
# for model_name in loader.models:
#   layer_name = loader.models[model_name][1][1]
#   f = all_p[model_name][layer_name][0]
#   y = all_p[model_name][layer_name][1]
#   if len(f.shape) > 2:
#     f = f.reshape(f.shape[0], -1)
#   score = hscore.get_hscore(f, y)
#   print(f'{model_name} {layer_name}: {score}.')
# print()

vgg19 fc2: 6.447368621826172.
resnet50 avg_pool: 11.661977767944336.
inception_v3 avg_pool: 4.620154857635498.
efficientnetb0 avg_pool: 24.094802856445312.
efficientnetb1 avg_pool: 23.421958923339844.
efficientnetb6 avg_pool: 22.185848236083984.



In [ ]:
# import hscore

# for i in range(5):
#   print(f'H scores for participant {i}:')
#   for model_name in loader.models:
#     layer_name = loader.models[model_name][1][0]
#     f = p[i].feats[model_name][layer_name]
#     y = p[i].encode_labels()
#     if len(f.shape) > 2:
#       f = f.reshape(f.shape[0], -1)
#     score = hscore.get_hscore(f, y)
#     print(f'{model_name} {layer_name}: {score}.')
#   print()

H scores for participant 0:
